Импорт пакетов

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from Bio import pairwise2
import json
from Bio.pairwise2 import format_alignment
from Bio import SeqIO

Введение функции учета качества выравнивания

In [2]:
def final_score(matches, all_symbols_in_seq, letters_in_seq, gaps_count_in_seq):

    y = (matches / all_symbols_in_seq) + (gaps_count_in_seq / letters_in_seq)
    return y 

Парсинг файлов

In [3]:
with open ('records\mutations_DS\just_mutations.json', 'r') as file:
    data = json.load(file)

    Sequence = data["AB032740"]["Protease"]["PR sequense"]

print(Sequence)

CCTCAAATCACTCTTTGGTCACGACCCTGGGTCACAGTACTTATAGGAGGACAGCTGTGGGAAGTGCTATTAGATAAAGGAGCAGATGATGGCGTATTAGAAGATATATGGTTGCCAGGAAAATGGAAACTAAGAATGATAGGGGGAATTGGAGGTTTTATCAAGGTAAGGCGTTATGATCAGATACTTATAGAAATTTGTGGAAAAAAGGCTATAGGTACAGTATTAGTAGGACCTACATCCGTCAACATAGTAGGACGAAATCACTTGACTAGGATTGGTTGTACTTATAATTTC


In [4]:
Reference = SeqIO.read('records\Test_records\HXB2.fasta', 'fasta').seq

print(Reference)

TGGAAGGGCTAATTCACTCCCAACGAAGACAAGATATCCTTGATCTGTGGATCTACCACACACAAGGCTACTTCCCTGATTAGCAGAACTACACACCAGGGCCAGGGATCAGATATCCACTGACCTTTGGATGGTGCTACAAGCTAGTACCAGTTGAGCCAGAGAAGTTAGAAGAAGCCAACAAAGGAGAGAACACCAGCTTGTTACACCCTGTGAGCCTGCATGGAATGGATGACCCGGAGAGAGAAGTGTTAGAGTGGAGGTTTGACAGCCGCCTAGCATTTCATCACATGGCCCGAGAGCTGCATCCGGAGTACTTCAAGAACTGCTGACATCGAGCTTGCTACAAGGGACTTTCCGCTGGGGACTTTCCAGGGAGGCGTGGCCTGGGCGGGACTGGGGAGTGGCGAGCCCTCAGATCCTGCATATAAGCAGCTGCTTTTTGCCTGTACTGGGTCTCTCTGGTTAGACCAGATCTGAGCCTGGGAGCTCTCTGGCTAACTAGGGAACCCACTGCTTAAGCCTCAATAAAGCTTGCCTTGAGTGCTTCAAGTAGTGTGTGCCCGTCTGTTGTGTGACTCTGGTAACTAGAGATCCCTCAGACCCTTTTAGTCAGTGTGGAAAATCTCTAGCAGTGGCGCCCGAACAGGGACCTGAAAGCGAAAGGGAAACCAGAGGAGCTCTCTCGACGCAGGACTCGGCTTGCTGAAGCGCGCACGGCAAGAGGCGAGGGGCGGCGACTGGTGAGTACGCCAAAAATTTTGACTAGCGGAGGCTAGAAGGAGAGAGATGGGTGCGAGAGCGTCAGTATTAAGCGGGGGAGAATTAGATCGATGGGAAAAAATTCGGTTAAGGCCAGGGGGAAAGAAAAAATATAAATTAAAACATATAGTATGGGCAAGCAGGGAGCTAGAACGATTCGCAGTTAATCCTGGCCTGTTAGAAACATCAGAAGGCTGTAGACAAATACTGGGACAGCTACAACCATCCCTTCAGACAG

Выравнивание и расчет качества

In [20]:
# стандартные значения параметров
openA = 0
extendA = 0
openB = 0
extendB = 0

for openB in range (-2, 0):
    for extendB in range (openB, 0):

        Alignment = pairwise2.align.localxd(Reference, Sequence, 
        openA, extendA, openB, extendB, 
        one_alignment_only=True)

        letters_in_seq = 0
        gaps_symbols_in_seq = 0
        gaps_count_in_seq = 0

        matches = pairwise2.align.localxd(Reference, Sequence, 
        openA, extendA, openB, extendB, 
        one_alignment_only=True, score_only = True)

        for symbol in Alignment[0].seqB: 
            if Alignment[0].seqB.index(symbol) in range(Alignment[0].start, Alignment[0].end):
                if symbol == '-':
                    gaps_symbols_in_seq += 1
                    if Alignment[0].seqB[Alignment[0].seqB.index(symbol) + 1] != '-':
                        gaps_count_in_seq += 1
                elif symbol != '-':
                    letters_in_seq += 1

        all_symbols_in_seq = letters_in_seq + gaps_symbols_in_seq

        Score = round(final_score(matches, all_symbols_in_seq, letters_in_seq, gaps_count_in_seq), 4)

        print(format_alignment(*Alignment[0]) + '\n' +
            'openA = ' + str(openA) + ' ' +
            'extendA = ' + str(extendA) + ' ' +
            'openB = ' + str(openB) + ' ' +
            'extendB = ' + str(extendB) + '\n')
        
        print('matches = ' + str(matches) +
         'all_symbols_in_seq = ' + str(all_symbols_in_seq) +
         'letters_in_seq = ' + str(letters_in_seq) +
         'gaps_count_in_seq = ' + str(gaps_count_in_seq) +
         'gaps_symbols_in_seq = ' + str(gaps_symbols_in_seq))
        
        print(Score)

2253 CCTCAGGTCACTCTTTGGCAACGACCCCTCGTCACAATAAAGATAGGGGGGCAACTAAAGGAAGCTCTATTAGATACAGGAGCAGATGATACAGTATTAGAAGAAATGAGTTTGCCAGGAAGATGGAAACCAAAAATGATAGGGGGAATTGGAGGTTTTATCAAAGTAAGACAGTATGATCAGATACTCATAGAAATCTGTGGACATAAAGCTATAGGTACAGTATTAGTAGGACCTACACCTGTCAACATAATTGGAAGAAATCTGTTGACTCAGATTGGTTGCACTTTAAATTT
     |||||..|||||||||||..|||||||...||||||.||...|||||.||.||.||...|||||..||||||||||.|||||||||||||...|||||||||||.||..|.||||||||||.||||||||.||.||||||||||||||||||||||||||||||.|||||.|..||||||||||||||.||||||||.||||||.|.||.||||||||||||||||||||||||||||||.|.|||||||||.|.|||.||||||..||||||..|||||||||.||||..|||||
   1 CCTCAAATCACTCTTTGGTCACGACCCTGGGTCACAGTACTTATAGGAGGACAGCTGTGGGAAGTGCTATTAGATAAAGGAGCAGATGATGGCGTATTAGAAGATATATGGTTGCCAGGAAAATGGAAACTAAGAATGATAGGGGGAATTGGAGGTTTTATCAAGGTAAGGCGTTATGATCAGATACTTATAGAAATTTGTGGAAAAAAGGCTATAGGTACAGTATTAGTAGGACCTACATCCGTCAACATAGTAGGACGAAATCACTTGACTAGGATTGGTTGTACTTATAATTT
  Score=245

openA = 0 extendA = 0 openB = -2 extendB = -2

matches = 245.0all_symbols_in_seq 

Расчет качества выравнивания

In [6]:
letters_in_seq = 0
gaps_symbols_in_seq = 0
gaps_count_in_seq = 0

matches = pairwise2.align.localxd(Reference, Sequence, 
openA, extendA, openB, extendB, 
one_alignment_only=True, score_only = True)

for symbol in Alignment[0].seqB:
    if symbol == '-':
        gaps_symbols_in_seq += 1
        if Alignment[0].seqB[Alignment[0].seqB.index(symbol) + 1] != '-':
            gaps_count_in_seq += 1
    elif symbol != '-':
        letters_in_seq += 1

all_symbols_in_seq = len(Alignment[0].seqB)

if all_symbols_in_seq != letters_in_seq + gaps_symbols_in_seq:
    print('Ошибка')

Score = round(final_score(matches, all_symbols_in_seq, letters_in_seq, gaps_count_in_seq), 4)

print(Score)

0.0306
